In [39]:
#import dependencies
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import requests
import json
from pprint import pprint
from datetime import datetime
from libs.usda import pull_usda_data

In [7]:
#assign usda api key
load_dotenv()
usda_api_key = os.getenv('usda_api_key')

In [34]:
#assign paramenter variables
# commodity_dict = {'CORN' : ['PRODUCTION', 'STOCKS'], 'SOYBEANS' : ['PRODUCTION', 'STOCKS']}

commodity = 'CORN'
statistic = 'PRODCUTION'
data_level = 'STATE'
state = 'NE'
year = 2020
data_columns = ['commodity_desc', 'statisticcat_desc', 'util_practice_desc', 'unit_desc', 'state_alpha', 'year', 'freq_desc', 'Value']